<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Class2-1" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-435: Class 2.1**
Welcome to Week 2! This week, we dive into all aspects of the Discrete Fourier Transform!



# Before we start: install `ipympl`

* This module is needed to enable interactive Matplotlib figures
* This installs packages that will require a restart of the runtime, so run this cell first.

In [ ]:
!pip install ipympl   # Also installs a more recent version of matplot (v3.5.3)

## The usual imports
Let's start by importing the "usual" modules we'll be using...

In [ ]:
import numpy as np                # Load the NumPy module, using the abbreviation 'np'.
import matplotlib.pyplot as plt   # Load the Matplotlib module, abbreviated as 'plt'
import IPython.display as ipd     # Load the Interactive Python display module, abbreviated as 'ipd'
import soundfile as sf            # Load the 'librosa' module for working with sound files and more

# Define a variable that's the directory path to our shared Google Drive folder
path = '/content/drive/MyDrive/eces435-work/class2.1/'

In [ ]:
# ENTER YOUR USERNAME
username = 'zyx321'

from google.colab import drive
drive.mount('/content/drive')

# Enable interactive Matplotlib figures

It's annoying that Colab's Matplotlib doesn't have this by default, but with a little work, we can enable basic figure interaction:
* Pan
* Zoom in/out
* Display of cursor coordinates

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
%matplotlib widget

## My plotting style defaults (optional)
Feel free to adjust these to your liking.

In [ ]:
from matplotlib import rc

rc('figure', figsize=(12,4))
rc('figure', facecolor='#aaaaaa')     # Better figure background for dark mode

rc('font', family='Liberation Serif') # Nicer font
rc('font', size=20)                   # Larger font size for labels

## `myPlot()` function
A quick time-domain signal plot function with my default figure settings and a time x-axis (in seconds).
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `N=#` Number of samples to plot (default: length of signal)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Plot format string (default: none)

In [ ]:
def myPlot(sig, N=0, fs=44100, fig_size=(16,4), x_ax=True, y_ax=True, lw=1, fmt=''):
  if N==0:
    N = len(sig)

  fig = plt.figure(figsize=fig_size)
  t = np.arange(N)/fs

  plt.plot( # Fill in this part )

  plt.xlabel('Time (sec)')
  ax = plt.gca()    # gca(): "Get current axis", the graph object that's currently plotted
  
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)

  fig.tight_layout()
  plt.ion()

  # Returning the figure causes issues with interactive matplotlib
  #return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

#Let's load up some music...

In [ ]:
filepath = path + 'aha.wav'
[aha, fs] = sf.read(filepath)
print('sampling rate:' , fs)

ipd.Audio(aha,rate=fs)

## Test out your `myPlot()` function
* Try zooming in / out
* Pan around the signal
* Save / download the figure

In [ ]:
myPlot(aha)

# Frequency analysis via the Fourier Series

Remember, according to Fourier, any signal can be composed out of a *sum of sinusoids* (frequencies).
* We've seen how sinusoids can be added to become *complex tones*.
* How do we do the *opposite*, meaning how do we determine the amount of a particular frequency within a signal? The *Fourier Series* gives us this:

  * $X(\omega) = \sum x[n] e^{-j \omega n} =\sum \left(\cos[\omega n] - j \sin[\omega n]\right)$
    * where $\omega = \frac{2 \pi f}{f_s}$, then...
  * $a = X_{Real}(\omega) = \sum x[n] \cos\left[\frac{2\pi f n}{f_s}\right]$
  * $b = -X_{Imag}(\omega) = \sum x[n] \sin\left[\frac{2\pi f}{f_s}\right]$
  * Magnitude $c = |X(\omega)| = \sqrt{a_k^2 + b_k^2}$

Write a function that takes a signal `x` and *one frequency* `f` as inputs and computes `a`, `b`, and `c`. 


In [ ]:
def frequencyAnalysis(x, f, fs=44100):
  n = # An array of sample indexes
  
  sin_f = np.sin(2*np.pi*f*n / fs)
  cos_f = np.cos(2*np.pi*f*n / fs)

  myPlot(x*cos_f, fmt='g')
  myPlot(x*sin_f, fmt='r')

  a = # Real part of Fourier series coefficient
  b = # Imaginary part of Fourier series coefficient
  c = # Magnitude of the Fourier series coefficient

  return a, b, c

Test out your `frequencyAnalysis` function

In [ ]:
a, b, c = frequencyAnalysis(aha, 100)
print(a, b, c)

## Create a simple bar graph for your Fourier Series magnitude coefficient
* Matplotlib's `bar` is a basic vertical bar graph

In [ ]:
def plotMag(f, c):
  fig = plt.figure(figsize=(12,8))
  plt.bar(f, c)
  plt.ylim(0,300)   # This range is kind of arbitrary, for now
  return fig

I will assign you a frequency for `my_f`

In [ ]:
my_f = # Input your assigned frequency

a, b, c = frequencyAnalysis(aha, my_f)
print(a, b, c)
figpath = path + 'freqs/' + str(my_f) + '-mag.png'
fig1 = plotMag(my_f, c)

# When you're ready, uncomment the next line to save the figure to Google Drive
#fig1.savefig(figpath)

# *==Pause here while everyone generates their figures==*

# The Fourier Series and Periodicity

Any signal can be composed out of a *sum of sinusoids* (frequencies), but the *Fourier Series* actually goes further:
* Every *periodic* signal is composed of sum of *harmonic* sinusoids (frequencies at integer multiples of a fundamental frequency, $f_0$).
* But the signal we just looked at *isn't periodic*! What did we do?!?!? 😱

Our previous frequency analysis was for the whole sample (~17 seconds).
* We just *assume* the full sample is our "period". What was the fundamental frequency, $f_0$?


In [ ]:
N = len(aha)  # total number of samples in our file
T0 = N / fs   # our "fundamental" period (in seconds)
print("Period:", T0, "seconds")
f0 = 1/T0     # our "fundamental" frequency
print("Fundamental:", f0, "Hz")

## Can we determine *when* a frequency happens?

Using the entire music sample doesn't tell us much about *when* those frequencies occur (or how they change over time).
* Instead, what if we take a short clip (20 ms) of the music? Let's call this an analysis *frame*.
* Then, we know any frequencies present in the frame occur within that short amount of time (20 ms).

How many samples give us 20 ms?

In [ ]:
dur = 0.02
dur * fs    # Number of samples?

Create a frame that we'll call `clip`.
* Have a listen (it's *really* short!)

In [ ]:
frameSize = int(fs * dur)   # We want to use this as an array index, so we cast it to int (otherwise you get an error)
clip = aha[fs : fs + frameSize]   # Let's clip a 0.02 sec segment, starting at 1 sec

myPlot(clip)
ipd.Audio(clip, rate=fs)

## Periodic extension
Remember, the Fourier Series requires an infinitely-long periodic signal. 

* Let's extend our frame periodidically to approximate what the Fourier Series is actually analyzing:

In [ ]:
clip_rep = np.tile(clip,50)
myPlot(clip_rep)

ipd.Audio(clip_rep,rate=fs)

## Now, what's our "fundamental" analysis frequency?

In [ ]:
T0 = frameSize / fs   # Our "fundamental" period hould be 20 ms
print("Period:",T0,"seconds")
f0 = # Fill in the relationship between frequency and period
print("Fundamental:",f0,"Hz")

# A straightforward implementation of the Fourier Transform
* The Fourier Transform is simply the Fourier Series of the periodic extension of our analysis *frame*.
  * Our fundamental analysis frequency, $f_0$ is determined by the number of samples in our frame (our "period").
  * The Fourier Series coefficients give us the amount of frequency at the harmonics of $f_0$.


  * $X[\omega_k] = \sum_{n=0}^{N-1} x[n] e^{-j \omega_k n} =\sum_{n=0}^{N-1} x[n] \left(\cos[\omega_k n] - j \sin[\omega_k n]\right)$
    * $\omega_k = 2 \pi f_k / f_s$
    * $N$ is the period (in samples), then…
  * $a_k = X_{Re}[\omega_k] = \sum_{n=0}^{N-1} x[n] \cos\left[\frac{2\pi f_k n}{f_s}\right]$
  * $b_k = -X_{Im}[\omega_k] = \sum_{n=0}^{N-1} x[n] \sin\left[\frac{2\pi f_k n}{f_s}\right]$
  * Magnitude $c_k = |X[\omega_k]| = \sqrt{a_k^2 + b_k^2}$

Write a function that takes a signal `x`, `fs`, and `K` (the number of Fourier coefficients to compute) as inputs and computes `a_k, b_k, and c_k`, the Fourier coefficients as outputs. 


In [ ]:
def myFourierTransform(x, fs, K=16):
  a_k = np.zeros(K)   # array for cos() weights
  b_k = np.zeros(K)   # array for sin() weights
  c_k = np.zeros(K)   # array for magnitude weights: sqrt(a**2 + b**2)

  fig = plt.figure(figsize = (20, 8))
  y_lim = 0.5;

  n = np.arange(len(x))

  for k in range(K):
    f_k =   # The current analysis frequency
    sin_k = 
    cos_k = 

    a_k[k] = 
    b_k[k] = 
    c_k[k] = 

    # Everything else in the loop is to make nice looking plots
    plt.subplot(3,K,k+1)      # Subplots are indexed starting at 1, a la MATLAB ?!?!?
    plt.plot(n,x*cos_k-0.1,'g')
    plt.fill_between(n,x*cos_k-0.1,-0.1,facecolor='g',alpha=0.5)
    plt.ylim(-y_lim,y_lim)
    plt.axis('off')
    
    plt.subplot(3,K,K + k+1)
    plt.plot(n,x,'c',n,0.1*cos_k+0.4 ,'g',n,0.1*sin_k-0.49,'r')
    plt.ylim(-0.6,0.6)
    plt.axis('off')

    plt.subplot(3,K,2*K + k+1)
    plt.plot(n,x*sin_k+0.1,'r')
    plt.fill_between(n,x*sin_k+0.1,0.1,facecolor='r',alpha=0.5)
    plt.ylim(-y_lim,y_lim)
    plt.axis('off')

  plt.show()

  # Plot the a and b arrays
  fig1 = plt.figure(figsize=(20,4))
  plt.bar(np.arange(K)-0.15, a_k, width=0.3,color='g')
  plt.bar(np.arange(K)+0.15, b_k, width=0.3,color='r')
  plt.show()

  # Plot the c array
  fig2 = plt.figure(figsize=(20,4))
  plt.bar(np.arange(K)*f0, c_k, width=30, color='b')
  plt.show()

  # Ouput c
  return a_k, b_k, c_k

## Let's use our Fourier Transform!

In [ ]:
a_k, b_k, c_k = myFourierTransform(clip, fs)

Print your `c_k` output for later comparison.

In [ ]:
c_k

##The (Discrete) Fourier Transform

Based on what we just did, this should look familiar:
* $X[k] = \sum_{n=0}^{N-1}x[n] e^\frac{-j 2 \pi f_k n}{N}$

Aside: Can you prove that using $\sin[2\pi f_k n]$ and $\cos[2\pi f_k n]$ is the same as $e^{-j2\pi f_k n}$ ?

In [ ]:
def myDFT(x, fs, K=16):
  X_k = np.zeros(K) * 1j    # Initialize the output as an array of complex numbers
  n = np.arange(len(x))

  for k in range(K):
    f_k = f0*k
    X_k[k] =  # Compute the Fourier output for f_k
              # Try using np.exp() instead of np.sin() and np.cos()

  return X_k

In [ ]:
C1 = myDFT(clip, fs)
np.abs(C1)

##Dude, isn't there an *easier* way to compute the Fourier transform?
Yes, the Fast Fourier Transform (FFT) is a *much* more efficient algorithm for computing the Discrete Fourier Transform (DFT). Lots of people say 'FFT' when they actually mean 'DFT'.

In [ ]:
C2 = np.fft.fft(clip)     # FFT: 'Fast Fourier Transform'
np.abs(C2[:16])

##What does the full DFT look like?

* Plot the real and imaginary outputs of the DFT.
* Add the magnitude of the DFT, $|X[k]|$?
* Separately, plot the magnitude in decibels (dB)?
  * $20 \log_{10} |X[k]|$

In [ ]:
N = len(clip)
fig = plt.figure(figsize=(20,4))
f = np.arange(N)*fs/N             # Frequency array, corresponding to Fourier frequencies (spaced at 50 Hz)

plt.plot(f, # Fill in this statement

#plt.xlim(0,fs/2)   # Try uncommenting this line

# But I *need* more frequency resolution!

Our frequency values are based on our "fundamental" period (the length of our frame), which is currently 50 Hz.
* What happens if you zero-pad (add zeros to the end of the signal)?
* Try different amounts of zero-padding

In [ ]:
N_z = 2048    # Zero-padded length of the frame
clip_z = np.append(clip, np.zeros(N_z - N)) # Zero-padded signal
f_z = np.arange(N_z) * fs / N_z   # Frequency vector (extended to zero-padded length)

C_z =  # Fill this in: the DFT of zero-padded signal

fig1 = plt.figure(figsize=(20,4))
plt.plot(clip_z)

fig2 = plt.figure(figsize=(20,4))
plt.plot(f_z, 20*np.log10(np.abs(C_z)))
plt.xlabel('Frequency (Hz)')
plt.xlim(0,fs/2)

## Periodic extension of a zero-padded frame

In [ ]:
clip_z_rep = np.tile(clip_z,15)
myPlot(clip_z_rep)
ipd.Audio(clip_z_rep,rate=fs)

## Analysis windows
Let's apply a window function to *taper* the edges of the analysis frame to reduce sidelobes in the frequency output. Some window functions (all built into NumPy):
* `hanning` (Hann window)
* `bartlett` (Triangle window)
* `hamming` (Raised cosine variation)
* `blackman` (High sidelobe reduction)

In [ ]:
clip_w = clip[:N] * np.hanning(N)       # Actually a 'Hann' function
#clip_w = clip[:N] * np.bartlett(N)     # Fancy name for a triangle
#clip_w = clip[:N] * np.hamming(N)      # Another sinusoidal window
#clip_w = clip[:N] * np.blackman(N)     # Another window with different tradeoffs

clip_wz = clip_z      # The whole frame (with zero-padding)
clip_wz[:N] = clip_w  # The windowed part of the frame

C_wz = np.fft.fft(clip_wz)

fig1 = plt.figure(figsize=(16,4))
plt.plot(clip_wz)

fig2 = plt.figure(figsize=(16,8))
#plt.plot(f, np.abs(X))
plt.plot(f_z, 20*np.log10(np.abs(C_wz)))

#plt.xlim(0,fs/2)
plt.xlim(0,10000)

In [ ]:
fig2.savefig(path + 'winDFT/' + username + '-winDFT.png')

#*==Pause here while everyone saves their figures==*

# Adding more DFT samples 
Adding more zeros increases the frequency resolution
* We do this so often, it's a parameter of the `fft` function.


In [ ]:
N_fft = 8192
C_w = np.fft.fft(clip_w)
C_wz = np.fft.fft(clip_w, n=N_fft)

fig = plt.figure(figsize=(16,4))
#ax = plt.axes(xlim=(-20,5020), ylim=(0, 90))

plt.plot(np.arange(N_fft)*fs/N_fft, 20*np.log10(np.abs(C_wz)),'.')
plt.plot(np.arange(N)*fs/N,20*np.log10(np.abs(C_w)),'.',markersize=15)
plt.xlim(0,5000)

#fig.savefig(path + 'Fourier Transform-windowed.png', dpi=200, transparent=True)

#Fourier Transform of a long signal

In [ ]:
AHA =           # DFT (FFT) of the full 'aha' signal
N =             # Number of samples in the 'aha' signal

fig1 = plt.figure()
n = np.arange(N)          # Sample index for signal
plt.plot(n/fs, aha)       # Plot the whole signal
plt.xlabel('Time (sec)')

fig2 = plt.figure()
plt.plot(np.arange(N)*fs/N, 20*np.log10(np.abs(AHA)))   # Plot the magnitude DFT
plt.xlim(0, fs/2)
plt.xlabel('Frequency (Hz)')

## myPlotFFT
Plot the magnitude frequency response (in dB FS) of a signal with my default figure settings and a frequency x-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `n_fft=#` The size of FFT to use (default: length of input signal)
  * `n_win=#` The length of Hann (hanning) window to use (default: length of input)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `x_lim=# or (#,#)` Frequency axis limits (max or range, in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Change plot formatting (default: none)

In [ ]:
def myPlotFFT(sig, fs=44100, n_fft=0, n_win=0, x_lim=0, fig_size=(16,4),x_ax=True, y_ax=True, lw=1, fmt=''):
  
  if n_fft==0:
    n_fft = len(sig)
  
  if n_win==0:
    n_win = len(sig)  
  win = np.hanning(n_win)

  S = # Fill this in

  N = len(S)
  f = np.arange(N) * fs / N
  S_mag = 4*np.abs(S) / n_fft     # Frequency magnitude, normalized by length
                                  #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                  #    x2 because Hanning window has 0.5 average
  S_mag += 0.0000001              # Add a small offset to avoid log(0) errors
  S_dBFS = 20*np.log10(S_mag)     # Freq. magnitude in dB full scale (dB FS):
                                  #    cos(w) -> 0 dBFS peak at w
  fig = plt.figure(figsize=fig_size)
  
  plt.plot( # Fill this in
           
  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = fs/2
    plt.xlim(0, x_lim)
  else:
    plt.xlim(x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB)')

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # Returning the figure causes issues with interactive matplotlib
  #return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

In [ ]:
myPlotFFT(aha)

#Let's make a (FFT) movie!

In [ ]:
# More modules required for animation
from matplotlib import animation, rc
from IPython.display import HTML

n_o = 0
f_size = int(fs*0.02)
n_hop = f_size / 2
N_fft = 4096
f = np.arange(N_fft) * fs / N_fft

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(14,6))
ax = plt.axes(xlim=(0,5000),ylim=(-100,100))
plt.close()   # Don't output the final figure separately

line, = ax.plot([], [])

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially  
def animate(i):
    n1 = int(n_o + n_hop*i)
    n2 = int(n_o + n_hop*i + f_size)

    x_i = ycagwyw[n1:n2]
    X_i = np.fft.fft(x_i * np.hanning(len(x_i)), n=N_fft)
    X_mag = 20*np.log(np.abs(X_i))

    line.set_data(f, X_mag)
    return (line,)  

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=50, interval=20, blit=True)

# Note: below is the part which makes it work on Colab
rc('animation', html='jshtml')
anim